1. Data Processing

In [1]:
# 1. ИМПОРТ БИБЛИОТЕК
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

from scipy import stats

from sklearn.impute import SimpleImputer


In [2]:
# Настройки отображения
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 10)
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

In [3]:
# Загрузка данных
# sales = pd.read_excel('Sales.xlsx', dtype={'OrderDate': str, 'ShipDate': str})
sales = pd.read_excel('Sales.xlsx')
customer = pd.read_excel('Customer.xlsx')
product = pd.read_excel('Product.xlsx')
territories = pd.read_excel('Territories.xlsx')

In [4]:
# Удалить строки, где **хотя бы одна** ячейка пустая
print(f"Строк после удаления пустых: {len(product)}")
product = product.dropna()

print(f"Осталось строк после удаления пустых: {len(product)}")

Строк после удаления пустых: 606
Осталось строк после удаления пустых: 331


In [5]:
# Заменяем все "NA" (как строку) на NaN
print(f"Осталось строк после удаления 'NA': {len(territories)}")
territories = territories.replace('NA', np.nan)

# Удаляем строки, где есть хотя бы одна ячейка с NaN
territories = territories.dropna()

print(f"Осталось строк после удаления 'NA': {len(territories)}")

Осталось строк после удаления 'NA': 11
Осталось строк после удаления 'NA': 10


In [6]:
# Консолидация данных
merged_data = sales.merge(customer, on='CustomerKey', how='left') \
                  .merge(product, on='ProductKey', how='left') \
                  .merge(territories, left_on='SalesTerritoryKey', right_on='SalesTerritoryKey', how='left')

# Проверка
print("Размер объединённого датасета:", merged_data.shape)
print("\nПервые 5 строк:")
print(merged_data.head())  

Размер объединённого датасета: (58189, 50)

Первые 5 строк:
   ProductKey  OrderDate   ShipDate  CustomerKey  PromotionKey  \
0         310 2014-01-01 2014-01-08        21768             1   
1         346 2014-01-01 2014-01-08        28389             1   
2         346 2014-01-01 2014-01-08        25863             1   
3         336 2014-01-01 2014-01-08        14501             1   
4         346 2014-01-01 2014-01-08        11003             1   

   SalesTerritoryKey SalesOrderNumber  SalesOrderLineNumber  OrderQuantity  \
0                  6          SO43697                     1              2   
1                  7          SO43698                     1              2   
2                  1          SO43699                     1              2   
3                  4          SO43700                     1              2   
4                  9          SO43701                     1              2   

   UnitPrice  TotalProductCost  SalesAmount    TaxAmt FirstName  LastName 

In [7]:
'''5. Печать информации о очищенных данных'''
print("\nИнформация об очищенных данных:")
print(merged_data.info())


Информация об очищенных данных:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58189 entries, 0 to 58188
Data columns (total 50 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   ProductKey            58189 non-null  int64         
 1   OrderDate             58189 non-null  datetime64[ns]
 2   ShipDate              58189 non-null  datetime64[ns]
 3   CustomerKey           58189 non-null  int64         
 4   PromotionKey          58189 non-null  int64         
 5   SalesTerritoryKey     58189 non-null  int64         
 6   SalesOrderNumber      58189 non-null  object        
 7   SalesOrderLineNumber  58189 non-null  int64         
 8   OrderQuantity         58189 non-null  int64         
 9   UnitPrice             58189 non-null  float64       
 10  TotalProductCost      58189 non-null  float64       
 11  SalesAmount           58189 non-null  float64       
 12  TaxAmt                58189 non-null  flo

In [8]:
'''6. Сохранение объединенных данных'''
merged_data.to_csv('10_merged_data.csv', index=False)

print("Объеденение данных завершено!")

Объеденение данных завершено!


In [9]:
# Дополнительные признаки
# Profit - чистая выручка разница Общая стоимость продукта и Сумма продаж
merged_data['Profit'] = merged_data['SalesAmount'] - merged_data['TotalProductCost']
# OrderYear год заказа
merged_data['OrderYear'] = merged_data['OrderDate'].dt.year
# OrderMonth месяц заказа
merged_data['OrderMonth'] = merged_data['OrderDate'].dt.month

print("Данные успешно загружены и подготовлены.")
print(f"Размер объединённого датасета: {merged_data.shape}")
print("\nПервые 5 строк:")
print(merged_data.head())

Данные успешно загружены и подготовлены.
Размер объединённого датасета: (58189, 53)

Первые 5 строк:
   ProductKey  OrderDate   ShipDate  CustomerKey  PromotionKey  \
0         310 2014-01-01 2014-01-08        21768             1   
1         346 2014-01-01 2014-01-08        28389             1   
2         346 2014-01-01 2014-01-08        25863             1   
3         336 2014-01-01 2014-01-08        14501             1   
4         346 2014-01-01 2014-01-08        11003             1   

   SalesTerritoryKey SalesOrderNumber  SalesOrderLineNumber  OrderQuantity  \
0                  6          SO43697                     1              2   
1                  7          SO43698                     1              2   
2                  1          SO43699                     1              2   
3                  4          SO43700                     1              2   
4                  9          SO43701                     1              2   

   UnitPrice  TotalProductCost  S

In [ ]:
# Удаление столбцов
# Посл анализа первичных данных удаляем дублирующиеся или не несущие ценности для анализа столбы
columns_to_drop = [
    'Photo',
    'ProductDescription',
    'NumberChildrenAtHome',
    'AddressLine1',
    'CustomerStateCode',
    'SalesOrderLineNumber',
    'StandardCost',  # равнозначен TotalProductCostпосле слиянияпосле слияния
    'ListPrice',     # равнозначен SalesAmount после слияния
    'PromotionKey',
    'CustomerCountry', # равнозначен Country после слияния
    'Group'
    'RegionImage',
    'RegionInfo',
]

merged_data = merged_data.drop(columns=columns_to_drop, errors='ignore')



In [11]:
'''5. Печать информации о очищенных данных'''
print("\nИнформация об очищенных данных:")
print(merged_data.info())


Информация об очищенных данных:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58189 entries, 0 to 58188
Data columns (total 43 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   ProductKey         58189 non-null  int64         
 1   OrderDate          58189 non-null  datetime64[ns]
 2   ShipDate           58189 non-null  datetime64[ns]
 3   CustomerKey        58189 non-null  int64         
 4   SalesTerritoryKey  58189 non-null  int64         
 5   SalesOrderNumber   58189 non-null  object        
 6   OrderQuantity      58189 non-null  int64         
 7   UnitPrice          58189 non-null  float64       
 8   TotalProductCost   58189 non-null  float64       
 9   SalesAmount        58189 non-null  float64       
 10  TaxAmt             58189 non-null  float64       
 11  FirstName          58189 non-null  object        
 12  LastName           58189 non-null  object        
 13  FullName           58189 non

In [12]:
'''1. Обработка пропусков'''
'''Печатаем количество пропущенных значений в каждом столбце'''
print("\nКоличество пропущенных значений:")
print(merged_data.isnull().sum())
print(merged_data.info())


Количество пропущенных значений:
ProductKey           0
OrderDate            0
ShipDate             0
CustomerKey          0
SalesTerritoryKey    0
                    ..
RegionImage          0
Region Info          0
Profit               0
OrderYear            0
OrderMonth           0
Length: 43, dtype: int64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58189 entries, 0 to 58188
Data columns (total 43 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   ProductKey         58189 non-null  int64         
 1   OrderDate          58189 non-null  datetime64[ns]
 2   ShipDate           58189 non-null  datetime64[ns]
 3   CustomerKey        58189 non-null  int64         
 4   SalesTerritoryKey  58189 non-null  int64         
 5   SalesOrderNumber   58189 non-null  object        
 6   OrderQuantity      58189 non-null  int64         
 7   UnitPrice          58189 non-null  float64       
 8   TotalProductCost   58189

In [13]:
merged_data.describe()

,ProductKey,OrderDate,ShipDate,CustomerKey,SalesTerritoryKey,OrderQuantity,UnitPrice,TotalProductCost,SalesAmount,TaxAmt,YearlyIncome,TotalChildren,HouseOwnerFlag,NumberCarsOwned,DaysToManufacture,StartDate,Profit,OrderYear,OrderMonth
count,58189.000000,58189,58189,58189.000000,58189.000000,58189.000000,58189.000000,58189.000000,58189.000000,58189.000000,58189.000000,58189.000000,58189.000000,58189.000000,30747.000000,30747,58189.000000,58189.000000,58189.000000
mean,437.208304,2016-06-03 03:56:09.605939200,2016-06-10 04:03:24.657237760,18853.004640,6.261716,1.569386,413.888218,296.539185,503.666270,40.293303,59769.887779,1.838921,0.690560,1.502466,1.978079,2007-04-01 09:01:43.619865344,207.127085,2015.868807,7.159257
min,214.000000,2014-01-01 00:00:00,2014-01-08 00:00:00,11000.000000,1.000000,1.000000,0.572500,0.856500,2.290000,0.183200,10000.000000,0.000000,0.000000,0.000000,0.000000,2005-07-01 00:00:00,1.433500,2014.000000,1.000000
25%,358.000000,2016-04-01 00:00:00,2016-04-08 00:00:00,14012.000000,4.000000,1.000000,4.990000,3.362300,8.990000,0.719200,30000.000000,0.000000,0.000000,1.000000,0.000000,2007-07-01 00:00:00,3.123700,2016.000000,4.000000
50%,479.000000,2016-07-07 00:00:00,2016-07-14 00:00:00,18151.000000,7.000000,1.000000,24.490000,12.192400,32.600000,2.608000,60000.000000,2.000000,1.000000,2.000000,0.000000,2007-07-01 00:00:00,15.643700,2016.000000,7.000000
75%,529.000000,2016-10-10 00:00:00,2016-10-17 00:00:00,23450.000000,9.000000,2.000000,269.995000,343.649600,539.990000,43.199200,80000.000000,3.000000,1.000000,2.000000,4.000000,2007-07-01 00:00:00,196.340400,2016.000000,10.000000
max,606.000000,2016-12-30 00:00:00,2017-01-07 00:00:00,29483.000000,10.000000,4.000000,3578.270000,2171.294200,3578.270000,286.261600,170000.000000,5.000000,1.000000,4.000000,4.000000,2007-07-01 00:00:00,1487.835600,2016.000000,12.000000
std,118.099746,NaN,NaN,5433.374315,2.960248,1.047532,833.052938,560.171436,941.462817,75.317027,33128.041818,1.614467,0.462267,1.155496,1.999912,NaN,384.285233,0.435668,3.284287


In [14]:
# Преобразование числовых данных
merged_data['UnitPrice'] = pd.to_numeric(merged_data['UnitPrice'], errors='coerce')
merged_data['TotalProductCost'] = pd.to_numeric(merged_data['TotalProductCost'], errors='coerce')
merged_data['SalesAmount'] = pd.to_numeric(merged_data['SalesAmount'], errors='coerce')
merged_data['TaxAmt'] = pd.to_numeric(merged_data['TaxAmt'], errors='coerce')
merged_data['YearlyIncome'] = pd.to_numeric(merged_data['YearlyIncome'], errors='coerce')


In [16]:
'''3. Приведение типов'''
'''Приведение столбца '*Date*' к типу datetime'''
# merged_data['OrderDate'] = pd.to_datetime(merged_data['OrderDate'], errors='coerce')
# merged_data['ShipDate'] = pd.to_datetime(merged_data['ShipDate'], errors='coerce')
# merged_data['BirthDate'] = pd.to_datetime(merged_data['BirthDate'], errors='coerce')

date_columns = [col for col in merged_data.columns if 'Date' in col]
for col in date_columns:
    merged_data[col] = pd.to_datetime(merged_data[col], errors='coerce').dt.date

In [19]:
'''5. Печать информации о очищенных данных'''
print("\nИнформация об очищенных данных:")
print(merged_data.info())
merged_data.describe()



Информация об очищенных данных:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58189 entries, 0 to 58188
Data columns (total 43 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   ProductKey         58189 non-null  int64  
 1   OrderDate          58189 non-null  object 
 2   ShipDate           58189 non-null  object 
 3   CustomerKey        58189 non-null  int64  
 4   SalesTerritoryKey  58189 non-null  int64  
 5   SalesOrderNumber   58189 non-null  object 
 6   OrderQuantity      58189 non-null  int64  
 7   UnitPrice          58189 non-null  float64
 8   TotalProductCost   58189 non-null  float64
 9   SalesAmount        58189 non-null  float64
 10  TaxAmt             58189 non-null  float64
 11  FirstName          58189 non-null  object 
 12  LastName           58189 non-null  object 
 13  FullName           58189 non-null  object 
 14  BirthDate          58189 non-null  object 
 15  MaritalStatus      58189 non-null  ob

,ProductKey,CustomerKey,SalesTerritoryKey,OrderQuantity,UnitPrice,TotalProductCost,SalesAmount,TaxAmt,YearlyIncome,TotalChildren,HouseOwnerFlag,NumberCarsOwned,DaysToManufacture,Profit,OrderYear,OrderMonth
count,58189.000000,58189.000000,58189.000000,58189.000000,58189.000000,58189.000000,58189.000000,58189.000000,58189.000000,58189.000000,58189.000000,58189.000000,30747.000000,58189.000000,58189.000000,58189.000000
mean,437.208304,18853.004640,6.261716,1.569386,413.888218,296.539185,503.666270,40.293303,59769.887779,1.838921,0.690560,1.502466,1.978079,207.127085,2015.868807,7.159257
std,118.099746,5433.374315,2.960248,1.047532,833.052938,560.171436,941.462817,75.317027,33128.041818,1.614467,0.462267,1.155496,1.999912,384.285233,0.435668,3.284287
min,214.000000,11000.000000,1.000000,1.000000,0.572500,0.856500,2.290000,0.183200,10000.000000,0.000000,0.000000,0.000000,0.000000,1.433500,2014.000000,1.000000
25%,358.000000,14012.000000,4.000000,1.000000,4.990000,3.362300,8.990000,0.719200,30000.000000,0.000000,0.000000,1.000000,0.000000,3.123700,2016.000000,4.000000
50%,479.000000,18151.000000,7.000000,1.000000,24.490000,12.192400,32.600000,2.608000,60000.000000,2.000000,1.000000,2.000000,0.000000,15.643700,2016.000000,7.000000
75%,529.000000,23450.000000,9.000000,2.000000,269.995000,343.649600,539.990000,43.199200,80000.000000,3.000000,1.000000,2.000000,4.000000,196.340400,2016.000000,10.000000
max,606.000000,29483.000000,10.000000,4.000000,3578.270000,2171.294200,3578.270000,286.261600,170000.000000,5.000000,1.000000,4.000000,4.000000,1487.835600,2016.000000,12.000000


In [20]:
print(merged_data.head())

   ProductKey   OrderDate    ShipDate  CustomerKey  SalesTerritoryKey  \
0         310  2014-01-01  2014-01-08        21768                  6   
1         346  2014-01-01  2014-01-08        28389                  7   
2         346  2014-01-01  2014-01-08        25863                  1   
3         336  2014-01-01  2014-01-08        14501                  4   
4         346  2014-01-01  2014-01-08        11003                  9   

  SalesOrderNumber  OrderQuantity  UnitPrice  TotalProductCost  SalesAmount  \
0          SO43697              2  1789.1350         2171.2942    3578.2700   
1          SO43698              2  1699.9950         1912.1544    3399.9900   
2          SO43699              2  1699.9950         1912.1544    3399.9900   
3          SO43700              2   349.5491          413.1463     699.0982   
4          SO43701              2  1699.9950         1912.1544    3399.9900   

     TaxAmt FirstName  LastName           FullName   BirthDate MaritalStatus  \
0  286

In [21]:
'''6. Сохранение очищенных данных'''
merged_data.to_csv('11_merged_cleaned_data.csv', index=False)

print("Очистка данных завершена!")

Очистка данных завершена!


Не используемые в коде части

In [ ]:
'''2. Удаление дубликатов'''
duplicates_count = data_cleaned.duplicated().sum()
print(f"\nКоличество дубликатов: {duplicates_count}")
data_cleaned = data_cleaned.drop_duplicates()

In [ ]:
'''1. Удаление строк/столбцов'''
df_dropped_rows = df.dropna()
df_dropped_cols = df.dropna(axis=1)

In [ ]:
'''Удаление строк с пропущенными значениями'''
data_cleaned = data.dropna()

In [ ]:
'''2. Заполнение средними значениями'''
mean_imputer = SimpleImputer(strategy='mean')
df_mean_filled = pd.DataFrame(mean_imputer.fit_transform(df), columns=df.columns)

нормализация и преобразование признаков

In [ ]:
df = pd.read_csv('merged_data.csv')

In [ ]:
'''1. Тест Граббса (Grubbs' Test)'''
def grubbs_test(data):
    N = len(data)
    mean = np.mean(data)
    std_dev = np.std(data)
    G = max(abs(data - mean)) / std_dev
    critical_value = stats.t.ppf(1 - 0.05 / (2 * N), N - 2)  # для 5% уровня значимости
    return G, G > critical_value

grubbs_result = grubbs_test(data)
print("Grubbs' Test: G =", grubbs_result[0], "Is outlier?", grubbs_result[1])

In [ ]:
'''2. Тест Диксона (Dixon's Q Test)'''
def dixon_test(data):
    data_sorted = np.sort(data)
    Q_low = (data_sorted[1] - data_sorted[0]) / (data_sorted[-1] - data_sorted[0])
    Q_high = (data_sorted[-1] - data_sorted[-2]) / (data_sorted[-1] - data_sorted[0])
    critical_value = 0.5  # Порог для Q (исходя из таблиц для n = 10)
    return Q_low > critical_value or Q_high > critical_value

dixon_result = dixon_test(data)
print("Dixon's Q Test: Is outlier?", dixon_result)

In [ ]:
'''4. Тест Розенбаума (Rosner's Test)'''
def rosner_test(data, max_outliers=1):
    N = len(data)
    data_sorted = np.sort(data)
    for i in range(max_outliers):
        mean = np.mean(data_sorted)
        std_dev = np.std(data_sorted)
        threshold = mean + (i + 1) * std_dev
        if data_sorted[-(i + 1)] > threshold:
            return True
    return False

rosner_result = rosner_test(data)
print("Rosner's Test: Is outlier?", rosner_result)

In [ ]:
'''5. Тест Тьо-Гентера (Tietjen-Moore Test)'''
def tietjen_moore_test(data):
    N = len(data)
    mean = np.mean(data)
    std_dev = np.std(data)
    z_scores = (data - mean) / std_dev
    return np.any(np.abs(z_scores) > 3)  # Порог Z-значения

tietjen_moore_result = tietjen_moore_test(data)
print("Tietjen-Moore Test: Is outlier?", tietjen_moore_result)

In [ ]:
'''6. Тест Стьюдента (Student's t-test)'''
t_statistic, p_value = stats.ttest_1samp(data, 0)
print("Student's t-test: t-statistic =", t_statistic, ", p-value =", p_value)

In [ ]:
'''7. Тест Спирмена (Spearman's Rank Correlation Coefficient)'''
spearman_corr, spearman_p = stats.spearmanr(data, np.arange(len(data)))
print("Spearman's Test: Correlation =", spearman_corr, ", p-value =", spearman_p)

In [ ]:
'''8. Тест Манна-Уитни (Mann-Whitney U Test)'''
# Для примера создадим два набора данных
data1 = np.array([1, 2, 3, 4, 5])
data2 = np.array([5, 6, 7, 8, 9])
mann_whitney_result = stats.mannwhitneyu(data1, data2)
print("Mann-Whitney U Test: U-statistic =", mann_whitney_result.statistic, ", p-value =", mann_whitney_result.pvalue)

In [ ]:
'''9. Тест Шапиро-Уилка (Shapiro-Wilk Test)'''
shapiro_stat, shapiro_p = stats.shapiro(data)
print("Shapiro-Wilk Test: W-statistic =", shapiro_stat, ", p-value =", shapiro_p)

In [ ]:
'''10. Тест Колмогорова-Смирнова (Kolmogorov-Smirnov Test)'''
ks_stat, ks_p = stats.kstest(data, 'norm', args=(np.mean(data), np.std(data)))
print("Kolmogorov-Smirnov Test: D-statistic =", ks_stat, ", p-value =", ks_p)

In [ ]:
'''3. Критерий Шовене (Chauvenet's Criterion)'''
def chauvenet_test(data):
    N = len(data)
    mean = np.mean(data)
    std_dev = np.std(data)
    d = abs(data - mean) / std_dev
    p_values = 1 / (2 * N * np.exp(0.5 * (d**2)))
    return np.any(p_values < 0.5)

chauvenet_result = chauvenet_test(data)
print("Chauvenet's Criterion: Is outlier?", chauvenet_result)